In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Set image dimensions and path
IMG_HEIGHT, IMG_WIDTH = 128, 128
DATA_PATH = '/content/drive/MyDrive/Signature'

def load_images():
    images, labels = [], []
    for category in ['genuine', 'forged']:
        path = os.path.join(DATA_PATH, category)
        for filename in os.listdir(path):
            # Parse the filename for labeling
            id_owner = filename.split('-')[1][:3]
            id_signer = filename.split('-')[1][5:8]
            # print(filename.split('-')[1][:3])
            # print(filename.split('-')[1][5:8])
            label = 1 if id_owner == id_signer else 0  # Genuine if owner == signer

            # Read and preprocess image
            # print('File name ',filename)
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            img = img / 255.0  # Normalize

            images.append(img)
            labels.append(label)

    images = np.array(images).reshape(-1, IMG_HEIGHT, IMG_WIDTH, 1)  # Reshape for CNN
    labels = np.array(labels)
    return images, labels

# Load data
X, y = load_images()
y = to_categorical(y, num_classes=2)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

Training set shape: (210, 128, 128, 1), Testing set shape: (45, 128, 128, 1)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Binary classification output
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,322 (12.60 MB)

 Trainable params: 3,304,322 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
EPOCHS = 45
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9430 - loss: 0.2203 - val_accuracy: 0.8222 - val_loss: 0.4603
Epoch 2/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 997ms/step - accuracy: 0.9297 - loss: 0.2134 - val_accuracy: 0.8444 - val_loss: 0.4506
Epoch 3/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9470 - loss: 0.1610 - val_accuracy: 0.8222 - val_loss: 0.5005
Epoch 4/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 873ms/step - accuracy: 0.9560 - loss: 0.1658 - val_accuracy: 0.8444 - val_loss: 0.4490
Epoch 5/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9582 - loss: 0.1399 - val_accuracy: 0.8667 - val_loss: 0.5137
Epoch 6/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 875ms/step - accuracy: 0.9809 - loss: 0.0893 - val_accuracy: 0.8444 - val_loss: 0.4072
Epoch 7/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9396 - loss: 0.1569 - val_accuracy: 0.8667 - val_loss: 0.4486
Epoch 8/45
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9544 - loss: 0.1020 - val_accuracy: 0.8667 - val_loss: 0.3640
Ep

In [6]:
model.save('/content/drive/MyDrive/Dataset_CNN/Signature/Models/sign_45.h5')

In [7]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Test Accuracy: 82.22%
Test Loss: 0.5561


In [9]:
import numpy as np

# ... (your existing code) ...

# Get predicted and true classes for test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)  # Extract true classes from y_test

print("Unique values in y_true_classes:", np.unique(y_true_classes))
print("Unique values in y_pred_classes:", np.unique(y_pred_classes))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
Unique values in y_true_classes: [0 1]
Unique values in y_pred_classes: [0 1]


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=["Forged", "Genuine"]))

# Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", cm)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step
              precision    recall  f1-score   support

      Forged       0.83      0.83      0.83        23
     Genuine       0.82      0.82      0.82        22

    accuracy                           0.82        45
   macro avg       0.82      0.82      0.82        45
weighted avg       0.82      0.82      0.82        45

Confusion Matrix:
 [[19  4]
 [ 4 18]]


In [11]:
import cv2
import numpy as np

def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (128, 128))  # Resize to match training dimensions
    img = img / 255.0  # Normalize to [0, 1]
    img = img.reshape(1, 128, 128, 1)  # Add batch and channel dimensions
    return img


In [12]:
# Path to the new image you want to test
file_path = '/content/drive/MyDrive/Signature/forged/NFI-08804004.png'

# Preprocess the image
processed_img = preprocess_image(file_path)

# Predict using the trained model
prediction = model.predict(processed_img)
predicted_class = np.argmax(prediction)  # 0 for forged, 1 for genuine

# Print the result
if predicted_class == 1:
    print("The signature is Genuine.")
else:
    print("The signature is Forged.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The signature is Forged.
